In [124]:
import json
import os
import re
from pathlib import Path
import pandas as pd
from bs4 import BeautifulSoup
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
import html as ihtml

In [97]:
def read_jsons_into_dataframe(directory):
    temp_list_of_dfs = []
    directory = directory
    pathlist = Path(directory).rglob('*.json')
    for path in pathlist:
        with open(path) as f:
            json_data = pd.json_normalize(json.loads(f.read()))
        temp_list_of_dfs.append(json_data)
    combined_df = pd.concat(temp_list_of_dfs, ignore_index=True)
    return(combined_df)

In [99]:
df_1700s = read_jsons_into_dataframe('1700s')
df_1800s = read_jsons_into_dataframe('1800s')
df_1900s = read_jsons_into_dataframe('1900s')
df_2000s = read_jsons_into_dataframe('2000s')

In [100]:
df_1700s['label'] = 1700
df_1800s['label'] = 1800
df_1900s['label'] = 1900
df_2000s['label'] = 2000

In [101]:
merged_df = pd.concat([df_1700s, df_1800s, df_1900s, df_2000s])
print(merged_df.columns)

Index(['date_blocked', 'id', 'blocked', 'judges', 'court', 'date_filed',
       'download_url', 'source', 'local_path', 'html_lawbox', 'time_retrieved',
       'nature_of_suit', 'plain_text', 'html_with_citations', 'sha1',
       'date_modified', 'precedential_status', 'extracted_by_ocr',
       'citation_count', 'absolute_url', 'docket', 'html', 'resource_uri',
       'citation.state_cite_three', 'citation.federal_cite_two',
       'citation.resource_uri', 'citation.federal_cite_three',
       'citation.lexis_cite', 'citation.document_uris',
       'citation.scotus_early_cite', 'citation.federal_cite_one',
       'citation.case_name', 'citation.westlaw_cite',
       'citation.state_cite_one', 'citation.specialty_cite_one',
       'citation.state_cite_regional', 'citation.id', 'citation.docket_number',
       'citation.state_cite_two', 'citation.neutral_cite',
       'supreme_court_db_id', 'label'],
      dtype='object')


In [102]:
import numpy as np
merged_df_of_interest = merged_df[['date_filed', 'plain_text', 'html', 'html_with_citations', 'label']]
merged_df_of_interest = merged_df_of_interest.replace(r'^\s*$', np.nan, regex=True)
print(merged_df_of_interest.shape)
merged_df_of_interest = merged_df_of_interest.dropna(subset=['plain_text', 'html', 'html_with_citations'], how='all')
print(merged_df_of_interest.shape)
#filter = merged_df_of_interest['html'] != ''
#filtered_df = df[filter]
#filtered_df = filtered_df.dropna()

(63374, 5)
(61314, 5)


In [103]:
print(merged_df_of_interest['html'].isna().sum())

1427


In [116]:
def clean_html_text(html_text):
    if html_text == '':
        pass 
    else:
        cleaned_text = BeautifulSoup(html_text, 'lxml').get_text()
    return(cleaned_text)

In [125]:
def clean_text(html_text):
    if html_text == '':
        pass
    else:
        text = BeautifulSoup(ihtml.unescape(html_text), "lxml").text
        text = re.sub(r"http[s]?://\S+", "", html_text)
        text = re.sub(r"\s+", " ", html_text)
    return text

In [129]:
def strip_html(html):
    if html == '':
        pass
    else:
        soup = BeautifulSoup(html, "html.parser")
        for data in soup(['style', 'script', 'code', 'a']):
            data.decompose()
    return ' '.join(soup.stripped_strings)

In [130]:
m_df = merged_df_of_interest.copy()
#m_df.head()
print(m_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61314 entries, 1 to 28383
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   date_filed           61314 non-null  object
 1   plain_text           984 non-null    object
 2   html                 59887 non-null  object
 3   html_with_citations  61268 non-null  object
 4   label                61314 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.8+ MB
None


In [131]:
m_df.head()

,date_filed,plain_text,html,html_with_citations,label
1,1773-09-01,NaN,"<p class=""case_cite"">1 U.S. 20</p>\r\n <p c...","<p class=""case_cite""><span class=""citation no-...",1700
2,1773-04-01,NaN,"<p class=""case_cite"">1 U.S. 18</p>\r\n <p c...","<p class=""case_cite""><span class=""citation no-...",1700
3,1787-06-01,NaN,"<p class=""case_cite"">1 U.S. 194</p>\n <p cl...","<div>\n<center><b><span class=""citation no-lin...",1700
4,1787-09-01,NaN,"<p class=""case_cite"">1 U.S. 210</p>\r\n <p ...","<div>\n<center><b><span class=""citation no-lin...",1700
5,1787-04-01,NaN,"<p class=""case_cite"">2 U.S. 97</p>\n <p cla...","<p class=""case_cite""><span class=""citation no-...",1700


In [135]:
def html_parser(raw_html):
    raw_html = str(raw_html)
    soup = BeautifulSoup(raw_html, 'html.parser')
    soup_string = soup.get_text()
    soup_string = re.sub('<.*>', ' ', soup_string)
    return soup_string

In [137]:
m_df['cleaned_text'] = m_df['html_with_citations'].progress_apply(lambda x: html_parser(x) if x != '' else x)

  0%|          | 0/61314 [00:00<?, ?it/s]

In [ ]:
m_df.to_json('/Users/kspicer/Desktop/spacy_textcat/cleaned_dataframe.jsonl', orient='records', lines=True)

In [ ]:
#_df = merged_df_of_interest.copy()
#mask = (m_df['plain_text'] != '') & (m_df['html'] != '') & (m_df['html_with_citations'] != '')

In [ ]:
#training_df = merged_df_of_interest[(merged_df_of_interest['plain_text'] != '') |
                        (merged_df_of_interest['html'] != '') | merged_df['html_with_citations'] 

KeyError: "['date_filed' 'plain_text' 'html' 'html_with_citations' 'label'] not found in axis"

In [ ]:
merged_df_of_interest.to_json('/Users/kspicer/Desktop/spacy_textcat/cleaned_dataframe.jsonl', orient='records', lines=True)

In [ ]:
merged_df_of_interest.iloc[0]

date_filed             1773-09-01
plain_text                       
html                             
html_with_citations              
label                        1700
Name: 0, dtype: object

In [ ]:
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()



/var/folders/jt/_w07f41j0zz__hb4jz9bz0vh0000gp/T/ipykernel_44520/1343133270.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


  0%|          | 0/63374 [00:00<?, ?it/s]

TypeError: argument of type 'NoneType' is not iterable

In [ ]:
merged_df_of_interest['cleaned_html'] = tqdm(merged_df_of_interest['cleaned_html'].replace(r'\n',' ', regex=True)) 

In [ ]:
dataframe_for_training = merged_df_of_interest[['cleaned_html', 'label']]
dataframe_for_training.json(' /Users/kspicer/Desktop/spacy_textcat/cleaned_dataframe.jsonl', orient='records', lines=True)

In [ ]:
'''
import re
import string
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import nltk

from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import word_tokenize

from sklearn.utils import shuffle
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


# some text cleaning functions
def convert_to_lower(text):
    return text.lower()

def remove_numbers(text):
    number_pattern = r'\d+'
    without_number = re.sub(pattern=number_pattern, repl=" ", string=text)
    return without_number

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_stopwords(text):
    removed = []
    stop_words = list(stopwords.words("english"))
    tokens = word_tokenize(text)
    for i in range(len(tokens)):
        if tokens[i] not in stop_words:
            removed.append(tokens[i])
    return " ".join(removed)

def remove_extra_white_spaces(text):
    single_char_pattern = r'\s+[a-zA-Z]\s+'
    without_sc = re.sub(pattern=single_char_pattern, repl=" ", string=text)
    return without_sc
'''

[nltk_data] Downloading package punkt to /Users/kspicer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/kspicer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kspicer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
'''
dataframe_for_training['cleaned_html'] = tqdm(dataframe_for_training['cleaned_html'].apply(lambda x: convert_to_lower(x)))
dataframe_for_training['cleaned_html'] = dataframe_for_training['cleaned_html'].apply(lambda x: remove_numbers(x))
dataframe_for_training['cleaned_html'] = dataframe_for_training['cleaned_html'].apply(lambda x: remove_punctuation(x))
dataframe_for_training['cleaned_html'] = dataframe_for_training['cleaned_html'].apply(lambda x: remove_stopwords(x))
dataframe_for_training['cleaned_html'] = dataframe_for_training['cleaned_html'].apply(lambda x: remove_extra_white_spaces(x))
'''

  0%|          | 0/63374 [00:00<?, ?it/s]

/var/folders/jt/_w07f41j0zz__hb4jz9bz0vh0000gp/T/ipykernel_23083/2176783017.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_for_training['cleaned_html'] = tqdm(dataframe_for_training['cleaned_html'].apply(lambda x: convert_to_lower(x)))
/var/folders/jt/_w07f41j0zz__hb4jz9bz0vh0000gp/T/ipykernel_23083/2176783017.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_for_training['cleaned_html'] = dataframe_for_training['cleaned_html'].apply(lambda x: remove_numbers(x))
/var/folders/jt/_w